In [ ]:
import os
from brayns import Client
from circuitexplorer import CircuitExplorer
from bioexplorer import BioExplorer, Metabolism
%matplotlib notebook

URL = 'localhost:5000'

be = BioExplorer(URL)
core = be.core_api()
be.reset_scene()

ce = CircuitExplorer(core)
data_folder = os.getenv('METABOLISM_DATA_FOLDER')


## Data loading

In [ ]:
load_neuron = True
load_astrocyte = True
load_vasculature = False
use_sdf = True

In [ ]:
if load_neuron:
    print('Loading neuron')
    config = '/gpfs/bbp.cscs.ch/project/proj129/medias/circuits/o1/CircuitConfig'
    neuron = ce.load_circuit(
        use_sdf_soma=use_sdf, 
        load_afferent_synapses=True, load_efferent_synapses=True,
        load_soma=True, load_axon=True, load_apical_dendrite=True, load_dendrite=True,
        circuit_color_scheme=ce.CIRCUIT_COLOR_SCHEME_NONE,
        morphology_color_scheme=ce.MORPHOLOGY_COLOR_SCHEME_NONE, generate_internals=True,
        path=config, gids=[170234 + 1]
    )
    neuron_model_id = neuron['id']

if load_astrocyte:
    print('Loading astrocyte...')
    config = '/gpfs/bbp.cscs.ch/project/proj129/medias/metabolism/astrocytes/one.astrocytes'
    astrocytes = ce.load_astrocytes(
        path=config, generate_internals=True, use_sdf_soma=use_sdf)
    astrocyte_model_id = astrocytes['id']

if load_vasculature:
    print('Loading vasculature...')
    config = '/gpfs/bbp.cscs.ch/project/proj129/caches/neuroscience/ngv/vasculature.brayns'
    vasculature = core.add_model(path=config, name='Vasculature')
    vasculature_model_id = vasculature['id']


In [ ]:
def set_metabolism_materials(model_id, user_param, color_1, color_2, color_3):
    colors = list()
    user_params = list()
    material_ids = ce.get_material_ids(model_id)['ids']
    print(material_ids)
    for material_id in material_ids:
        mid = material_id % ce.NB_MATERIALS_PER_MORPHOLOGY
        if mid in [ce.MATERIAL_OFFSET_AFFERENT_SYNPASE, ce.MATERIAL_OFFSET_EFFERENT_SYNPASE]: # Synapses
            colors.append(color_2)
            user_params.append(3)
        elif mid == ce.MATERIAL_OFFSET_MITOCHONDRION: # Mitochondria
            colors.append(color_3)
            user_params.append(user_param)
        else: 
            colors.append(color_1)
            user_params.append(user_param)
       
    ce.set_material_extra_attributes(model_id)
    ce.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        user_parameters=user_params
    )
    
if load_neuron:
    set_metabolism_materials(neuron_model_id, 0, [0,1,1], [1,1,1], [1,0,1])
if load_astrocyte:
    set_metabolism_materials(astrocyte_model_id, 1, [1,1,0], [0,0,0], [0,0,1])
if load_vasculature:
    set_metabolism_materials(vasculature_model_id, 7, [0,0,0], [0,0,0], [0,0,0])

### Camera

In [ ]:
status = core.set_camera(
    orientation=[0.351, 0.039, 0.033, 0.934],
    position=[459.479, 681.489, 932.141],
    target=[439.152, 817.321, 776.323]
)

# Widgets

In [ ]:
from sqlalchemy import create_engine

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = os.getenv('DB_SCHEMA')

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

generate_snapshot = False
generate_movie = False

In [ ]:
from bioexplorer import Metabolism

simulation_timestamp = '3'
metabolism = Metabolism(
    be, neuron_model_id, db_host, db_name, db_user, db_password, db_schema, simulation_timestamp)
metabolism.display()

In [ ]:
metabolism.set_renderer(
    subsampling=1, max_accum_frames=4, alpha_correction=0.5, 
    noise_frequency=5.0, noise_amplitude=0.05,
    ray_step=3.0, search_length=20.0, use_random_search=False,
    exposure=1.0, far_plane=200.0)

## Snapshot

In [ ]:
from bioexplorer import BioExplorer, MovieMaker

be = BioExplorer(URL)
mm = MovieMaker(be)
output_folder = '/tmp'

In [ ]:
if generate_snapshot:
    mm.create_snapshot(
        size=[960, 540], samples_per_pixel=16,
        path=output_folder, base_name='metabolism_1k_v1')

## Movie

In [ ]:
from IPython.display import clear_output
from bioexplorer import MovieMaker
mm = MovieMaker(be)

In [ ]:
key_frames = [
{'apertureRadius': 0.0,
 'direction': [-0.024215636912091935, 0.6397371067346861, -0.7682122344740901],
 'focusDistance': 1000000.0,
 'origin': [454.7949205096349, 690.6436543447036, 928.330436921203],
 'up': [-0.035184191178338306, 0.7674161301314157, 0.6401832205745853]},
{'apertureRadius': 0.0,
 'direction': [-0.02421563691209099, 0.6397371067346889, -0.7682122344740879],
 'focusDistance': 1000000.0,
 'origin': [468.14540061429307, 746.2337081741207, 870.6722615031416],
 'up': [-0.03518419117833822, 0.7674161301314134, 0.6401832205745881]}
]

In [ ]:
# if True:
if generate_movie:
    k = 1
    nb_frames = 360
    mm.build_camera_path(key_frames, nb_frames, 10)
    nb_frames = mm.get_nb_frames()
    print('Number of frames: %d' % nb_frames)
    
    output_folder = '/tmp'

    from IPython.display import clear_output
    for frame in range(nb_frames):
        clear_output()
        print('Frame %d' % frame)
        mm.set_current_frame(frame)
        brayns.set_animation_parameters(current=frame)

        mm.create_snapshot(
            size=[k * 960, k * 540], samples_per_pixel=8,
            path=output_folder, base_name='%05d' % frame)

In [ ]:
mm.cancel_frames_export()